In [1]:

import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 2
n_layer = 3
dropout = 0.2

In [3]:
#set manual seed for future reproducibility
torch.manual_seed(100)

In [ ]:
#!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt


In [4]:
path = "input.txt" 
with open(path, 'r') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)


# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

decode(encode("hello world"))

'hello world'

In [5]:
# create a dataset that reads input.txt and encodes it as a sequence of integers. each batch returns sequnce of size block_size
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, text, block_size, transform):
        self.transform = transform    
        self.text = text
        self.block_size = block_size # here are all the unique characters that occur in this text
        self.chars = sorted(list(set(text)))
      
    def __len__(self):
        return len(self.text) - self.block_size - 1

    def __getitem__(self, idx):
        return self.transform(self.text[idx:idx+self.block_size]) , self.transform(self.text[idx+1:idx+self.block_size+1])


# define transform function that takes string list of characters and returns a tensor of integers
def transform(x):
    # convert list of characters to list of integers
    x = encode(x)
    return torch.tensor(x, dtype=torch.long)

#divide text into train and val
train_data = text[:int(len(text)*0.9)]
val_data = text[int(len(text)*0.9):]

train_dataset = TextDataset(train_data, block_size, transform)
val_dataset = TextDataset(val_data, block_size, transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

print(next(iter(train_loader)))



[tensor([[27, 30, 32,  ..., 30, 15, 37],
        [31, 43, 41,  ..., 57, 54, 43],
        [39, 52, 52,  ..., 19, 24, 27],
        ...,
        [ 1, 15, 50,  ..., 59, 56, 57],
        [ 1, 57, 43,  ...,  1, 58, 46],
        [57,  1, 13,  ..., 15, 27, 30]]), tensor([[30, 32, 20,  ..., 15, 37, 10],
        [43, 41, 53,  ..., 54, 43, 41],
        [52, 52, 53,  ..., 24, 27, 33],
        ...,
        [15, 50, 39,  ..., 56, 57,  1],
        [57, 43, 52,  ..., 58, 46, 43],
        [ 1, 13, 52,  ..., 27, 30, 21]])]


In [6]:
def estimate_loss(model , dataloader):
    model.eval()
    total_loss = 0
    for x,y in dataloader:
        x = x.to(device)
        y = y.to(device)
        with torch.no_grad():
            logits = model(x)
            loss = F.cross_entropy(logits.transpose(1,2), y)
            total_loss += loss.item()
    return total_loss / len(dataloader)


In [7]:
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, x):
        return self.emb(x)
    
    def generate(self , idx, max_new_tokens):
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [8]:
s = "my name is shubham"
s = transform(s).to(device)
s = s.unsqueeze(0)

context = torch.zeros((1,1) , dtype=torch.long).to(device)
print(decode(s.tolist()[0]))
print(decode(model.generate(s, max_new_tokens=200)[0].tolist()))


my name is shubham


NameError: name 'model' is not defined

In [10]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
        
        return logits

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [11]:
model = BigramLanguageModel().to(device)
#print number of parameters
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

5468993


In [12]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

iterator = iter(train_loader)

for i in range(max_iters):
    model.train()
    x , y = next(iterator)
    x = x.to(device)
    y = y.to(device)
    optimizer.zero_grad()
    logits = model(x)
    loss = F.cross_entropy(logits.transpose(1,2), y)
    loss.backward()
    optimizer.step()
    
    #compuate loss after every eval_interval
    if i % eval_interval == 0:
        train_loss = 0 # estimate_loss(model, train_loader)
        val_loss = estimate_loss(model, val_loader)
        print("iter: {} | train_loss: {:.4f} | val_loss: {:.4f}".format(i, train_loss, val_loss))

iter: 0 | train_loss: 0.0000 | val_loss: 3.8036
iter: 500 | train_loss: 0.0000 | val_loss: 2.0783


KeyboardInterrupt: 